In [1]:
import tensorflow as tf

In [2]:
def my_image_filter(input_images):
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(input_images, conv1_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)

    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)

In [4]:
img1 = tf.placeholder(tf.float32,shape=(100,32,32,32))

In [5]:
result1 = my_image_filter(img1)

In [6]:
result2 = my_image_filter(img1)

    Problem is that you call my_image_filter twice,but this will create two sets of variables,4 variable in each one, for a total of 8 variables

In [21]:
variables_dict = {
    "conv1_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),name="conv1_weights"),
    "conv1_biases": tf.Variable(tf.zeros([32]), name="conv1_biases"),
    "conv2_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),name="conv2_weights"),
    "conv2_biases": tf.Variable(tf.zeros([32]), name="conv2_biases")}

In [22]:
def my_image_filter(input_images, variables_dict):
    conv1 = tf.nn.conv2d(input_images, variables_dict["conv1_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + variables_dict["conv1_biases"])

    conv2 = tf.nn.conv2d(relu1, variables_dict["conv2_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + variables_dict['conv2_biases'])

In [23]:
# Both calls to my_image_filter() now use the same variables
result1 = my_image_filter(img1, variables_dict)
result2 = my_image_filter(img1, variables_dict)

With convenient ,creating varibles like above , outside of the code, breaks encapsulation
* the code builds graph must document the names ,shapes of variable to create.
* when the code changes ,the callers may have to create more , less or different varibles

### Variable Scope mechanism that allows to easily share named variables while constructing a graph

In [24]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [25]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

In [26]:
result1 = my_image_filter(img1)
result2 = my_image_filter(img1)

ValueError: Variable conv1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1228, in __init__
    self._traceback = _extract_stack()
  File "C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2336, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 768, in apply_op
    op_def=op_def)


In [29]:
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(img1)
    scope.reuse_variables()
    result2 = my_image_filter(img1)

1. The scope is set for creating new variables ,as evidenced by tf.get_variable_scope.reuse() = false

In [ ]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
assert v.name == "foo/v:0"

In [40]:
v

<tf.Variable 'foo/bar/v:0' shape=(1,) dtype=float32_ref>

2.the scope is set for reusing variables as evidenced by tf.get_variable_scope.reuse()=True

In [33]:
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 is v

The primary function of variable scope is to carry a name that will be used as **prefix** for variables names and a **reuse-flag** to distinguish the two cases discribed above.

In [34]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"

nesting variable scopes append their names in a way analogous to how directories work

the current scope can be retieved by using tf.get_variable_scope .and a reuse flag can be set to True by tf.get_variable_scope.reuse_variables()

In [42]:
with tf.variable_scope("foo"):
    tf.get_variable_scope().reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 is v

AssertionError: 

In [43]:
v1

<tf.Variable 'foo/v:0' shape=(1,) dtype=float32_ref>

you cannot set the reuse flag to false. but you can enter a reusing variable scope and then exit it, then exist it ,going back to a non_reusing one 
* using a paramster reuse = True when opening a variable scope
* the reusing parameter is inherited

In [44]:
with tf.variable_scope("root"):
    # At start, the scope is not reusing.
    assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo"):
        # Opened a sub-scope, still not reusing.
        assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo", reuse=True):
        # Explicitly opened a reusing scope.
        assert tf.get_variable_scope().reuse == True
        with tf.variable_scope("bar"):
            # Now sub-scope inherits the reuse flag.
            assert tf.get_variable_scope().reuse == True
    # Exited the reusing scope, back to a non-reusing one.
    assert tf.get_variable_scope().reuse == False

In [55]:
with tf.variable_scope("foo") as foo_scope:
    pass

In complex model ,It may be useful to pass a variablescope object rather than relying on the name right.

To this end, variable scopes can be captured and used instead of names when opening a new variable scope

In [57]:
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])

Error arise when:
* case 1: when variable_scope is set reuse=True  ,cannot get exist variable.
* case 2: when variable_scope is set reuse=False ,the new variable have been exist.

In [58]:
w

<tf.Variable 'foo/w:0' shape=(1,) dtype=float32_ref>

In [59]:
v

<tf.Variable 'foo/bar/v:0' shape=(1,) dtype=float32_ref>

In [60]:
with tf.variable_scope(foo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
assert v1 is v
assert w1 is w

AssertionError: 

When opening a variable scope using a previously existing scope 

we jump out of the current variable scope prefix to an entirely different one. This is fully independent of where we do it

In [61]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.

 set the default initializer for all variables in one place, on top of all functions

In [104]:
tf.reset_default_graph()
with tf.variable_scope("foo"):
    x = 1.0 + tf.get_variable("v", [1])
assert x.op.name == "foo/add"

when we do with tf.variable_scope('name') ,this implictly opens a tf.name_scope
* variable_scope to govern names of variable.
* name_scope only affect the the names of ops, but not of variables. 

In [105]:
def functionA(*a, **kw):
    print(a)
    print(kw)

functionA(1, 2, 3, 4, 5, 6, a=2, b=3, c=5)

(1, 2, 3, 4, 5, 6)
{'b': 3, 'c': 5, 'a': 2}


In [106]:
lis=[1, 2, 3, 4]
dic={'a': 10, 'b':20}
functionA(*lis, **dic)  #it is similar to functionA(1, 2, 3, 4, a=10, b=20)

(1, 2, 3, 4)
{'b': 20, 'a': 10}


In [107]:
*dic

SyntaxError: can't use starred expression here (<ipython-input-107-327b7a26b64c>, line 1)

In [12]:
tf.reset_default_graph()

In [13]:
w = tf.Variable(1)
mul = tf.multiply(w, 2)
add = tf.add(w, 2)
group = tf.group(mul, add)
tuple = tf.tuple([mul, add])

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [4]:
c = sess.run(group)

In [5]:
b = sess.run(tuple)

In [6]:
b

[2, 3]

In [7]:
add

<tf.Tensor 'Add:0' shape=() dtype=int32>

control dependencies context applies only to ops that are constructed within the context,merely using an op or tensor in the context does not add a control dependency, the following example illustrate this point

In [15]:
with tf.control_dependencies([mul]):
    # The matmul op is created outside the context, so no control
    # dependency will be added.
    mul = tf.multiply(tf.add(w,2),mul)

In [16]:
sess.run(mul)

6

In [18]:
'{0},{1}'.format('kzc',18)

'kzc,18'

Merge_all mechanism

In [181]:
tf.reset_default_graph()

if you want to input a batch of samples ,you need to define input as [None,_,_].None provide placeholder for batch size

In [182]:
v = tf.placeholder(dtype = tf.float32,shape =[None,1])

summary.scalar must be a numerical value, even batchsize has to be averaged

In [183]:
a = tf.add(v,1)
a_s=tf.summary.scalar('s1',tf.reduce_mean(a))

In [184]:
b = tf.add(v,2)
b_s=tf.summary.scalar('s2',tf.reduce_mean(b))

In [185]:
c = tf.add(v,3)
c_s=tf.summary.scalar('s3',tf.reduce_mean(c))

In [186]:
merge_all = tf.summary.merge_all()

In [173]:
merge_all

<tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>

In [174]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [175]:
sess.run(a,feed_dict={v:[[1],[2]]})

array([[ 2.],
       [ 3.]], dtype=float32)

In [177]:
a_m = sess.run(merge_all,feed_dict={v:[[1]]})

In [178]:
print(a_m)

b'\n\t\n\x02s1\x15\x00\x00\x00@\n\t\n\x02s2\x15\x00\x00@@\n\t\n\x02s3\x15\x00\x00\x80@'


Compute gradient

In [14]:
x = tf.placeholder(dtype=tf.float32,shape=[1])
z = tf.placeholder(dtype=tf.float32,shape=[1])

In [12]:
w = tf.Variable(initial_value=tf.random_uniform([1],-1,1),name='w')
b = tf.Variable(initial_value=tf.zeros([1]),name='b',dtype=tf.float32)

In [13]:
y= w*x+b

In [16]:
loss = tf.reduce_mean(tf.square(x-z))

In [19]:
optim = tf.train.AdadeltaOptimizer(0.5)

In [20]:
grad =optim.compute_gradients(loss,var_list=[w,b])

In [25]:
for gradient,variable in grad:
    print(variable.name)
    print(variable.dtype)
    print(variable)

w_1:0
<dtype: 'float32_ref'>
<tf.Variable 'w_1:0' shape=(1,) dtype=float32_ref>
b_1:0
<dtype: 'float32_ref'>
<tf.Variable 'b_1:0' shape=(1,) dtype=float32_ref>


about device

In [3]:
import tensorflow as tf

In [4]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# 新建session with log_device_placement并设置为True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# 运行这个 op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


# Write to file

In [4]:
with open('name.txt','w') as writer:
    lines = ['line 1\n',
            'this is line 2\n'
            'this is line 3\n'
            'this is end']
    writer.writelines(lines)

In [6]:
file_text = open('name.txt','r')
print(file_text.read())

line 1
this is line 2
this is line 3
this is end


In [10]:
value = ['the answer', 42]
s = str(value)

In [9]:
print(s)

('the answer', 42)


In [12]:
from datetime import datetime

In [13]:
'{:%Y-%m-%d %H:%M}'.format(datetime(2001, 2, 3, 4, 5))

'2001-02-03 04:05'

In [4]:
import numpy as np

In [5]:
charar = np.chararray((3, 3))

In [6]:
charar[:] = 'a'

In [7]:
charar

chararray([[b'a', b'a', b'a'],
       [b'a', b'a', b'a'],
       [b'a', b'a', b'a']], 
      dtype='|S1')

# Defaultdict

In [46]:
from collections import defaultdict

In [48]:
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]

In [49]:
d = defaultdict(list)

In [50]:
d

defaultdict(list, {})

In [51]:
for k, v in s:
    d[k].append(v)

In [54]:
type(d)

collections.defaultdict

In [12]:
sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

list:

In [14]:
d = {}

In [16]:
type(d)

dict

In [17]:
for k, v in s:
    d.setdefault(k, []).append(v)

In [18]:
sorted(d.items())

[('blue', [2, 4]), ('red', [1]), ('yellow', [1, 3])]

In [19]:
d

{'blue': [2, 4], 'red': [1], 'yellow': [1, 3]}

In [22]:
d.items()

dict_items([('red', [1]), ('yellow', [1, 3]), ('blue', [2, 4])])

In [23]:
s = 'mississippi'

In [24]:
d = defaultdict(int)

In [25]:
for k in s:
    d[k] += 1

In [26]:
d

defaultdict(int, {'i': 4, 'm': 1, 'p': 2, 's': 4})

In [27]:
def constant_factory(value):
    return lambda: value

In [28]:
d = defaultdict(constant_factory('<missing>'))

In [29]:
d

defaultdict(<function __main__.constant_factory.<locals>.<lambda>>, {})

In [30]:
d.update(name='John', action='ran')

In [31]:
'%(name)s %(action)s to %(object)s' % d

'John ran to <missing>'

In [32]:
s = [('red', 1), ('blue', 2), ('red', 3), ('blue', 4), ('red', 1), ('blue', 4)]

In [33]:
d = defaultdict(set)

In [34]:
for k, v in s:
    d[k].add(v)

In [35]:
d

defaultdict(set, {'blue': {2, 4}, 'red': {1, 3}})

In [36]:
sorted(d.items())

[('blue', {2, 4}), ('red', {1, 3})]

In [37]:
type(d)

collections.defaultdict

In [38]:
dishes = {'eggs': 2, 'sausage': 1, 'bacon': 1, 'spam': 500}

In [41]:
dishes.items()

dict_items([('bacon', 1), ('sausage', 1), ('spam', 500), ('eggs', 2)])

In [43]:
dishes.values()

dict_values([1, 1, 500, 2])

In [44]:
n = 0
for val in values:
    n += val
print(n)

504


In [45]:
list(dishes.items())

[('bacon', 1), ('sausage', 1), ('spam', 500), ('eggs', 2)]

In [58]:
import _pickle

PANDAS

In [68]:
import pandas as pd
import numpy as np

In [69]:
df3 = pd.DataFrame(np.random.randn(10, 5),columns=['a', 'b', 'c', 'd', 'e'])

In [70]:
df3

,a,b,c,d,e
0,-2.393344,0.275463,-0.345161,-0.885426,-0.126815
1,-0.752567,1.674186,0.735057,0.741968,-1.206360
2,-1.152015,-1.697202,-0.635909,-0.559359,1.814964
3,0.478055,-0.737784,1.550367,2.322799,-0.405939
4,0.126121,1.144100,-0.169250,-1.557432,0.549155
5,-0.645509,-0.848838,-1.449495,-0.527453,-1.121423
6,-2.178385,-0.547616,-0.690179,-1.550765,0.142442
7,-0.220253,-1.239641,0.411852,-1.036674,0.026550
8,0.858310,0.196444,0.920189,-0.728645,-0.602052
9,-0.592051,0.806812,0.609923,-0.581334,0.022644


In [71]:
df2 = pd.DataFrame(np.random.randn(10, 5))

In [72]:
df2

,0,1,2,3,4
0,0.160411,-0.574837,-0.229245,-0.823229,0.155549
1,-0.120446,1.043990,-0.445605,0.117430,0.834774
2,-0.465894,0.137532,-0.278313,1.647168,0.187978
3,0.757789,-0.324990,2.082582,0.445584,1.349514
4,-0.288145,-1.505305,0.101736,-1.143095,1.210972
5,-0.864318,0.355775,0.354493,0.568237,1.637264
6,0.349829,-0.371160,1.913420,0.392538,-1.187562
7,1.712015,0.161036,-1.179671,0.168751,-0.757411
8,0.161739,-0.000109,-0.570597,-0.619769,0.442973
9,0.682682,0.232505,0.556509,-1.151741,1.204022
